In [2]:
# import libs
import kaggle
import polars as pl

In [3]:
# download dataset
dataset = 'anandshaw2001/netflix-movies-and-tv-shows'
path = f'C:/Users/pedro/Documents/Kaggle/{dataset}'
kaggle.api.dataset_download_files(
    dataset, 
    path=path,
    unzip=True)

Dataset URL: https://www.kaggle.com/datasets/anandshaw2001/netflix-movies-and-tv-shows


In [4]:
# load to polars abordagem eager
file = 'netflix_titles.csv'
df = pl.read_csv(
    f'{path}/{file}'
)

# load to polars abordagem lazy
df_lazy = pl.scan_csv(
    f'{path}/{file}'
)

In [5]:
# simple manipulation (df / eager)
df2 = df \
    .group_by('country') \
    .agg([
        pl.len().alias('count'),
        pl.median('release_year').alias('release_year_median')
    ])
df2

country,count,release_year_median
str,u32,f64
"""United Kingdom,""",2,2017.0
"""Uruguay, Argentina, Spain""",1,2018.0
"""Germany, Jordan, Netherlands""",1,2016.0
"""United Kingdom, Germany, Unite…",1,2020.0
"""United States, United Kingdom,…",1,2009.0
…,…,…
"""United States, Italy, Romania,…",1,2003.0
"""Chile, France""",1,2016.0
"""United States, Senegal""",1,2015.0


In [6]:
# simple manipulation (sql / lazy)
df3 = df_lazy.sql(
    'select type, count(*) from self group by type'
)
df3.collect()

type,len
str,u32
"""TV Show""",2676
"""Movie""",6131
